In [1]:
from circuit import gen_random_circuits
from circuit.formatter import qiskit_to_my_format_circuit, layered_circuits_to_qiskit
from upstream import RandomwalkModel
from collections import defaultdict

import math
from utils.backend import gen_grid_topology, get_grid_neighbor_info, Backend, gen_linear_topology, get_linear_neighbor_info
# from utils.backend import default_basis_single_gates, default_basis_two_gates

from qiskit import QuantumCircuit
import pennylane as qml
import numpy as np
from downstream.synthesis.pennylane_op import layer_circuit_to_pennylane_circuit, layer_circuit_to_pennylane_tape
from downstream.synthesis.tensor_network_op import layer_circuit_to_matrix
from downstream.synthesis.synthesis_model_dnn import matrix_distance_squared

from scipy.stats import unitary_group

from downstream.synthesis.synthesis_model_dnn_unitary import pkl_dump, pkl_load
from downstream.synthesis.synthesis_model_dnn_unitary import SynthesisModel

import time
from qiskit import transpile
import random
# gird topological information
# grid_num = 2
# topology = gen_grid_topology(grid_num) # 3x3 9 qubits
# neigh_info = get_grid_neighbor_info(grid_num, 1)
# n_qubits = grid_num ** 2


# n_qubits = 2
# layer2gates = [
#     [
#         {'name': 'u', 'qubits': [0], 'params': [np.pi]*3},
#         {'name': 'u', 'qubits': [0], 'params': [np.pi]*3},
#     ],
#     [{'name': 'cz', 'qubits': [1,0], 'params': []}],
#     [
#         {'name': 'u', 'qubits': [0], 'params': [np.pi]*3},
#         {'name': 'u', 'qubits': [0], 'params': [np.pi]*3},
#     ],
#     [{'name': 'cz', 'qubits': [0,1], 'params': []}],
#     [
#         {'name': 'u', 'qubits': [0], 'params': [np.pi]*3},
#         {'name': 'u', 'qubits': [0], 'params': [np.pi]*3},
#     ],
#     [{'name': 'cz', 'qubits': [0,1], 'params': []}],
#     [
#         {'name': 'u', 'qubits': [0], 'params': [np.pi]*3},
#         {'name': 'u', 'qubits': [0], 'params': [np.pi]*3},
#     ],
#     [{'name': 'cz', 'qubits': [1,0], 'params': []}],
#     [
#         {'name': 'u', 'qubits': [0], 'params': [np.pi]*3},
#         {'name': 'u', 'qubits': [0], 'params': [np.pi]*3},
#     ],
# ]
# U = unitary_group.rvs(2**n_qubits)
# params = SynthesisModel.find_parmas(n_qubits, layer2gates, U, max_epoch=100)


# 2-qubit topological information
n_qubits = 4
topology = gen_linear_topology(n_qubits)
neigh_info = get_linear_neighbor_info(n_qubits, 1)


backend = Backend(n_qubits=n_qubits, topology=topology, neighbor_info=neigh_info, basis_single_gates=['u'],
                  basis_two_gates=['cz'], divide=False, decoupling=False)

model_name = f'q{n_qubits}_0220_dnn_unitary'

'''生成用于测试的模板电路'''
max_gate = 2**n_qubits
min_gate = 5
circuits = gen_random_circuits(min_gate=min_gate, max_gate=max_gate, gate_num_step=max(
    [1, (max_gate+max_gate)//50]), n_circuits=5, two_qubit_gate_probs=[3, 8], backend=backend, reverse=False, optimize=True, multi_process=True)
# upstream_model_path = f'./temp_data/upstream_model_q{n_qubits}_0220.pkl'
upstream_model = RandomwalkModel(
    1, 20, backend=backend, travel_directions=('parallel', 'next'))
upstream_model.train(circuits, multi_process=True)

# random.shuffle(upstream_model.dataset)
# circuits = upstream_model.dataset[:1000]
synthesis_model = SynthesisModel(
    upstream_model=upstream_model, name=model_name)
synthesis_model.construct_data(circuits=circuits, multi_process=True)
synthesis_model.construct_model()

2023-02-24 11:31:59,904	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


finish random circuit generation with 275 circuits
(remote_train pid=2059199) train:0/27, 81th offest
(remote_train pid=2061751) train:0/27, 216th offest
(remote_train pid=2059120) train:0/27, 108th offest
(remote_train pid=2059128) train:0/27, 162th offest
(remote_train pid=2059115) train:0/27, 189th offest
(remote_train pid=2059133) train:0/27, 27th offest
(remote_train pid=2059124) train:0/27, 243th offest
(remote_train pid=2059266) train:0/5, 270th offest
(remote_train pid=2059536) train:0/27, 0th offest
(remote_train pid=2059465) train:0/27, 54th offest
random walk finish device size =  7
0 path table size =  4
1 path table size =  7
2 path table size =  6
3 path table size =  6
(0, 1) path table size =  6
(1, 2) path table size =  6
(2, 3) path table size =  7
start generate Us -> gate vectos
(remote_train pid=2060733) train:0/27, 135th offest
finished rate = 0 / 275


(_gen_data_remote pid=2059115) WARNING:jax._src.lib.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_gen_data_remote pid=2059120) WARNING:jax._src.lib.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_gen_data_remote pid=2060733) WARNING:jax._src.lib.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_gen_data_remote pid=2059266) WARNING:jax._src.lib.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_gen_data_remote pid=2059536) WARNING:jax._src.lib.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_gen_data_remote pid=2059465) WARNING:jax._src.lib.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_gen_data_remote pid=2059128) WARNING:jax._src.lib.xla_bridge:N

len(Us) =  19350 len(gate_vecs) =  19350
start construct model
Iteration 1, loss = 0.52016355
Iteration 2, loss = 0.33117137
Iteration 3, loss = 0.21306956
Iteration 4, loss = 0.15581974
Iteration 5, loss = 0.12656694
Iteration 6, loss = 0.11098788
Iteration 7, loss = 0.10244455
Iteration 8, loss = 0.09701373
Iteration 9, loss = 0.09227252
Iteration 10, loss = 0.08760237
Iteration 11, loss = 0.08288684
Iteration 12, loss = 0.07843174
Iteration 13, loss = 0.07458937
Iteration 14, loss = 0.07122143
Iteration 15, loss = 0.06838153
Iteration 16, loss = 0.06621775
Iteration 17, loss = 0.06453808
Iteration 18, loss = 0.06315600
Iteration 19, loss = 0.06219795
Iteration 20, loss = 0.06156612
Iteration 21, loss = 0.06105611
Iteration 22, loss = 0.06060222
Iteration 23, loss = 0.06024665
Iteration 24, loss = 0.05993474
Iteration 25, loss = 0.05963073
Iteration 26, loss = 0.05936099
Iteration 27, loss = 0.05921950
Iteration 28, loss = 0.05916745
Iteration 29, loss = 0.05896419
Iteration 30, loss